# Setting the Palette

In [1]:
# First, load the codes that convert between RGB and HSL codes
!curl -O https://raw.githubusercontent.com/Leejere/python-visualization-preset/main/convert_colors.ipynb

%run convert_colors.ipynb

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 10665  100 10665    0     0   278k      0 --:--:-- --:--:-- --:--:--  289k


## Setting up the palette dataframe

In this document, we set the basic palette table as the following table.

| Category | Role | Regular Variation | Faded Variation | Dimmed Variation |
| --- | --- | --- | --- | --- |
| MAIN | Hero | *violate* | *purple* | *dk violet* |
| MAIN | primary | *salmon* | *lt salmon* | *dk salmon* |
| MAIN | Highlight | *lemon* | *canary* | *citrine* |
| MAIN | Green | *sea green* | *lt sea green* | *dk sea green* |
| MAIN | Water | *turquoise* | *lt turquoise* | *dk turquoise*|
| BACKUP | Backup1 | *orange* | *lt orange* | *dk orange*|
| BACKUP | Backup2 | *forest* | *lt forest* | *dk forest*|
| BACKUP | Backup3 | *red* | *lt red* | *dk red* |

A few things to note:
- Color categories. The **Main Palette** contains the primary colors of this color system. The colors in this category are supposed to work well together and may be used in large areas. The **Backup** colors are used in categorical plotting or mapping when there aren't enough colors to use.
- Color variation. Three variations exist for each *Role*: *Regular*, *Faded*, and *Dimmed*. The faded variations have higher lightness and are used to assist the regular colors and to add hierarchy. On the other hand, the dimmed versions are darker and less saturated and are better with dark or black backgrounds.
- How to generate the colors. The *Regular* colors are generated by input. The *Faded* and *Dimmed* versions are calculated based on the regular versions.
- Naming and indexing. Colors are not indexed by *color names* (for example, "red", "green", or "salmon") but are referred to by *Role* and *Variation*, e.g., Hero Regular, Highlight Faded, or Water Dimmed. In this way, codes can remain unchanged and readable even if we change the entire color palette.

We use a Pandas DataFrame to store the color values. Below, we create an empty dataframe to store color values.

In [2]:
# Setting indexes
import numpy as np
import pandas as pd

category = pd.Series(['main', 'main', 'main', 'main', 'main', 'backup', 'backup', 'backup'])
role = pd.Series(['hero', 'primary', 'highlight', 'green', 'water', 'backup1', 'backup2', 'backup3'])

# Empty dataframe
palette = pd.DataFrame(np.empty((8, 3), dtype = str),
                       columns = ['regular', 'faded', 'dimmed'], 
                       index = [category, role])

palette = palette.applymap(lambda x: '#FFFFFF') # Let's fill them with whites for now.
palette

regular    faded   dimmed
main   hero       #FFFFFF  #FFFFFF  #FFFFFF
       primary    #FFFFFF  #FFFFFF  #FFFFFF
       highlight  #FFFFFF  #FFFFFF  #FFFFFF
       green      #FFFFFF  #FFFFFF  #FFFFFF
       water      #FFFFFF  #FFFFFF  #FFFFFF
backup backup1    #FFFFFF  #FFFFFF  #FFFFFF
       backup2    #FFFFFF  #FFFFFF  #FFFFFF
       backup3    #FFFFFF  #FFFFFF  #FFFFFF

## Creating a demo function

In this section, we set up a demonstration function so that we can view our palette every step of the way.

In [3]:
from ipycanvas import Canvas

zoom = 1.8

# Set the starting point of the canvas
ox = 10 * zoom
oy = 10 * zoom

cell_width = 100 * zoom
cell_height = 25 * zoom

gap_x = 20 * zoom
gap_y = 10 * zoom

cell_kerning = cell_width + gap_x
cell_leading = cell_height + gap_y

category_column_kerning = 70 * zoom
role_column_kerning = 75 * zoom

text_offset_x = 3 * zoom
text_offset_y = 15 * zoom

def demonstrate_palette():
    # For the Dimmed column, draw a black background
    canvas.fill_style = '#000000'
    canvas.fill_rect(ox + category_column_kerning + role_column_kerning + cell_kerning * 2 - gap_x / 2,
                     0,
                     cell_kerning,
                     9 * cell_leading + oy
                    )
    
    # Print table head
    canvas.fill_style = '#000000'
    canvas.font = '28px sans-serif'

    canvas.fill_text('category', ox + text_offset_x, oy + text_offset_y)
    
    canvas.fill_text('role', ox + category_column_kerning + text_offset_x, oy + text_offset_y)
    current_category = ''
    
    for column in range(palette.shape[1]):
        if palette.columns[column] == 'dimmed':
            canvas.fill_style = '#FFFFFF' # The third column: name should be white against black background
        else:
            canvas.fill_style = '#000000'
        canvas.fill_text(palette.columns[column], 
                         ox + column * cell_kerning + category_column_kerning + role_column_kerning + text_offset_x, 
                         oy + text_offset_y)
        
    for row in range(palette.shape[0]):
        this_y = oy + (row + 1) * cell_leading # everything in this row shares y coord

        canvas.fill_style = '#000000'
        canvas.font = '28px sans-serif'

        # Print category name
        if current_category != palette.index[row][0]:
            # if new category, print it
            canvas.fill_text(palette.index[row][0], ox + text_offset_x, this_y + text_offset_y) 
            current_category = palette.index[row][0]

        # Print role name
        canvas.fill_text(palette.index[row][1], ox + category_column_kerning + text_offset_x, this_y + text_offset_y)

        # Print colors and their hex
        canvas.font = '22px sans-serif'
        for column in range(palette.shape[1]):
            # Where to draw this cell
            this_color = palette.iloc[row, column]
            this_x = ox + column * cell_kerning + category_column_kerning + role_column_kerning

            canvas.fill_style = this_color
            # Draw this color, starting from this point
            # Cell width depending on whether main or backup
            if current_category == 'backup':
                this_width = cell_width / 2
            else:
                this_width = cell_width
                
            canvas.fill_rect(this_x, this_y, this_width, cell_height)
            
            # Print the corresponding hex
            if(hex_to_hsl(this_color)[2] > 70):
                hex_text_color = '#000000'
            else:
                hex_text_color = '#FFFFFF'
            canvas.fill_style = hex_text_color
            canvas.fill_text(this_color, this_x + text_offset_x, this_y + text_offset_y)

## Filling in the colors

Now, we start to define our colors. First, we define our *Regular* colors.

In [4]:
# Now we start to define the colors
palette.regular = ['#353795', # hero, violet
                   '#ef8872', # primary, salmon
                   '#fdd310', # highlight, yellow
                   '#7cbfa4', # green, see green
                   '#84cbce', # water, turquoise
                   '#f4a422', # backup1, orange
                   '#5db75a', # backup2, forest
                   '#e54225' # backup3, red
                  ]

Next, we will create the *Fade* and *Dimmed* colors based on the *Regular* colors. First, we will create two function that turn the colors faded or dimmed.

In [5]:
# Function to fade a color. Takes a hex and the ratio of s/l increase, outputs a hex.
def make_faded(reg_hex, s_fade_ratio, l_fade_ratio):
    this_h, this_s, this_l = [i for i in hex_to_hsl(reg_hex)]
    faded_h = this_h
    faded_s = min(this_s * s_fade_ratio, 100)
    faded_l = min(this_l * l_fade_ratio, 100)
    return hsl_to_hex(faded_h, faded_s, faded_l)

# Function to dim a color. Takes a hex and the ratio of s/l decrease, outputs a hex.
def make_dimmed(reg_hex, s_dim_ratio, l_dim_ratio):
    this_h, this_s, this_l = [i for i in hex_to_hsl(reg_hex)]
    dimmed_h = this_h
    dimmed_s = this_s * s_dim_ratio
    dimmed_l = this_l * l_dim_ratio
    return hsl_to_hex(dimmed_h, dimmed_s, dimmed_l)

In [6]:
# Determine the fading or dimming extents
# Let's first try something
# Sequence: hero, primary, highlight, green, water, backup 1-3
s_fade_ratio = np.array([1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2])
l_fade_ratio = np.array([1.3, 1.3, 1.3, 1.3, 1.3, 1.3, 1.3, 1.3])

s_dim_ratio = np.array([0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8])
l_dim_ratio = np.array([0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8])

# Make a ratio table
ratio_table = pd.DataFrame({"regular": palette['regular'],
                                     "s_fade_ratio": s_fade_ratio,
                                     "l_fade_ratio": l_fade_ratio,
                                     "s_dim_ratio": s_dim_ratio,
                                     "l_dim_ratio": l_dim_ratio
                                    })
ratio_table

regular  s_fade_ratio  l_fade_ratio  s_dim_ratio  \
main   hero       #353795           1.2           1.3          0.8   
       primary    #ef8872           1.2           1.3          0.8   
       highlight  #fdd310           1.2           1.3          0.8   
       green      #7cbfa4           1.2           1.3          0.8   
       water      #84cbce           1.2           1.3          0.8   
backup backup1    #f4a422           1.2           1.3          0.8   
       backup2    #5db75a           1.2           1.3          0.8   
       backup3    #e54225           1.2           1.3          0.8   

                  l_dim_ratio  
main   hero               0.8  
       primary            0.8  
       highlight          0.8  
       green              0.8  
       water              0.8  
backup backup1            0.8  
       backup2            0.8  
       backup3            0.8

In [7]:
# Create Faded and Dimmed colors by passing the regular colors along with ratios
palette['faded'] = ratio_table.apply(lambda x: make_faded(x['regular'], x['s_fade_ratio'], x['l_fade_ratio']), axis = 1)
palette['dimmed'] = ratio_table.apply(lambda x: make_dimmed(x['regular'], x['s_dim_ratio'], x['l_dim_ratio']), axis = 1)

In [8]:
# Let's look at the current palette
canvas = Canvas(height = (palette.shape[0] + 1) * cell_leading + oy, 
                width = (palette.shape[1] + 1) * cell_kerning + ox + category_column_kerning + role_column_kerning,
                sync_image_data=True)
demonstrate_palette()

<img src="demonstrate/palette_raw.png"  width="550">

## Final adjustments

Some of the faded and dimmed colors are desirable. We may need to modify the `ratio_table` to make it not uniform, and even have to re-input some colors

In [9]:
# Re-make ratio_table
# Sequence: hero, primary, highlight, green, water, backup 1-3
s_fade_ratio = np.array([1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2])
l_fade_ratio = np.array([1.3, 1.25, 1.6, 1.35, 1.35, 1.4, 1.4, 1.4])

s_dim_ratio = np.array([0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8])
l_dim_ratio = np.array([0.8, 0.75, 0.7, 0.7, 0.8, 0.8, 0.8, 0.8])

# Make a ratio table
ratio_table = pd.DataFrame({"regular": palette['regular'],
                                     "s_fade_ratio": s_fade_ratio,
                                     "l_fade_ratio": l_fade_ratio,
                                     "s_dim_ratio": s_dim_ratio,
                                     "l_dim_ratio": l_dim_ratio
                                    })
# Create Faded and Dimmed colors by passing the regular colors along with ratios
palette['faded'] = ratio_table.apply(lambda x: make_faded(x['regular'], x['s_fade_ratio'], x['l_fade_ratio']), axis = 1)
palette['dimmed'] = ratio_table.apply(lambda x: make_dimmed(x['regular'], x['s_dim_ratio'], x['l_dim_ratio']), axis = 1)

In [10]:
# Our hero color needs a re-input
palette.loc[('main', 'hero'), 'faded'] = '#c8b7d9'

In [11]:
# Let's look at the current palette
canvas = Canvas(height = (palette.shape[0] + 2) * cell_leading + oy, 
                width = (palette.shape[1] + 1) * cell_kerning + ox + category_column_kerning + role_column_kerning,
                sync_image_data=True)
demonstrate_palette()

<img src="demonstrate/palette.png"  width="550">

Let's store individual colors by names so than we can use them easily.

In [12]:
palette = palette.reset_index().rename({"level_1": "role"}, axis = 1).drop(['level_0'], axis = 1).set_index(['role'])

In [13]:
# Names of individual colors & color series

for column in palette.columns:
    globals()[f"palette_{column}"] = palette[column].values
    for row in palette.index: 
        if column == "regular":
            globals()[f"palette_{row}"] = palette.loc[row, column]
        else:
            globals()[f"palette_{row}_{column}"] = palette.loc[row, column]

We are quite happy with this final result. So we'll keep it.

## Final code (to be used elsewhere)

In [1]:
# Libraries and existing codes
# First, load the codes that convert between RGB and HSL codes
!curl -O https://raw.githubusercontent.com/Leejere/python-visualization-preset/main/convert_colors.ipynb
%run convert_colors.ipynb
import numpy as np
import pandas as pd

# EMPTY DATAFRAME
category = pd.Series(['main', 'main', 'main', 'main', 'main', 'backup', 'backup', 'backup'])
role = pd.Series(['hero', 'primary', 'highlight', 'green', 'water', 'backup1', 'backup2', 'backup3'])

palette = pd.DataFrame(np.empty((8, 3), dtype = str),
                       columns = ['regular', 'faded', 'dimmed'], 
                       index = [category, role])

# FILL IN COLORS
'''
PLEASE INPUT VALUES HERE
'''
palette.regular = ['#353795', # hero, violet
                   '#ef8872', # primary, salmon
                   '#fdd310', # highlight, yellow
                   '#7cbfa4', # green, see green
                   '#84cbce', # water, turquoise
                   '#f4a422', # backup1, orange
                   '#5db75a', # backup2, forest
                   '#e54225' # backup3, red
                  ]

# FADE/DIM FUNCTION
def make_faded(reg_hex, s_fade_ratio, l_fade_ratio):
    this_h, this_s, this_l = [i for i in hex_to_hsl(reg_hex)]
    faded_h = this_h
    faded_s = min(this_s * s_fade_ratio, 100)
    faded_l = min(this_l * l_fade_ratio, 100)
    return hsl_to_hex(faded_h, faded_s, faded_l)


def make_dimmed(reg_hex, s_dim_ratio, l_dim_ratio):
    this_h, this_s, this_l = [i for i in hex_to_hsl(reg_hex)]
    dimmed_h = this_h
    dimmed_s = this_s * s_dim_ratio
    dimmed_l = this_l * l_dim_ratio
    return hsl_to_hex(dimmed_h, dimmed_s, dimmed_l)

# FADING OR DIMMING RATIOS
'''
PLEASE INPUT VALUES HERE
'''
s_fade_ratio = np.array([1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2])
l_fade_ratio = np.array([1.3, 1.25, 1.6, 1.35, 1.35, 1.4, 1.4, 1.4])

s_dim_ratio = np.array([0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8])
l_dim_ratio = np.array([0.8, 0.75, 0.7, 0.7, 0.8, 0.8, 0.8, 0.8])

# CREATE FINAL TABLE
ratio_table = pd.DataFrame({"regular": palette['regular'],
                                     "s_fade_ratio": s_fade_ratio,
                                     "l_fade_ratio": l_fade_ratio,
                                     "s_dim_ratio": s_dim_ratio,
                                     "l_dim_ratio": l_dim_ratio
                                    })
# Create Faded and Dimmed colors by passing the regular colors along with ratios
palette['faded'] = ratio_table.apply(lambda x: make_faded(x['regular'], x['s_fade_ratio'], x['l_fade_ratio']), axis = 1)
palette['dimmed'] = ratio_table.apply(lambda x: make_dimmed(x['regular'], x['s_dim_ratio'], x['l_dim_ratio']), axis = 1)

# FINAL ADJUSTMENT
palette.loc[('main', 'hero'), 'faded'] = '#c8b7d9'

# OUTPUT
palette = palette.reset_index().rename({"level_1": "role"}, axis = 1).drop(['level_0'], axis = 1).set_index(['role'])
# Names of individual colors & color series
for column in palette.columns:
    globals()[f"palette_{column}"] = palette[column].values
    for row in palette.index: 
        if column == "regular":
            globals()[f"palette_{row}"] = palette.loc[row, column]
        else:
            globals()[f"palette_{row}_{column}"] = palette.loc[row, column]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 10665  100 10665    0     0  70215      0 --:--:-- --:--:-- --:--:-- 70629


In [2]:
palette.to_csv("palette.csv", index_label = ['category', 'role'])